In [1]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from IPython import get_ipython
import os
import base64
from pyngrok import ngrok
import random
import librosa
# import eye_tracking
# import voice
# import voice_output

In [2]:
app = Flask(__name__)
CORS(app)

In [3]:
UPLOAD_FOLDER = '/home/jegathees5555/Documents/projects/AI_Interview_recruitz/backend/server/'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

In [4]:
def upload_video(video_data):        
   

    if video_data:
        video_name = 'uploaded_video.webm'
        video_path = os.path.join(app.config['UPLOAD_FOLDER'], video_name)
        print(video_path)
        print(video_name)
        video_data.save(video_path)  # Save the file
        
        
        
        # return result
        return jsonify({'message': 'Video uploaded successfully'}), 200
    else:
        return jsonify({'error': 'No video data provided'}), 400

In [5]:
from concurrent.futures import ThreadPoolExecutor
import cv2
import numpy as np
from flask import jsonify
from tensorflow.keras.models import load_model

2024-01-26 18:34:44.395537: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-26 18:34:44.400241: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-26 18:34:44.504066: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-26 18:34:44.505325: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-26 18:34:46.026894: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [6]:
emotion_model = load_model('/home/jegathees5555/Documents/projects/AI_Interview_recruitz/backend/facial_emotion/imageclassifier.h5')

In [7]:
def eye_tracking_method(video_path):
    # eye_detector = cv2.CascadeClassifier("/home/jegathees5555/Documents/recruitz/backend/eye_track/haarcascade_eye.xml")
    cascade_classifiers = [
        cv2.CascadeClassifier("/home/jegathees5555/Documents/recruitz/backend/eye_track/haarcascade_eye.xml"),
        cv2.CascadeClassifier("/home/jegathees5555/Documents/projects/AI_Interview_recruitz/backend/eye_track/haarcascade_frontalface_default.xml")
        # Add your second Haar Cascade classifier here if needed
    ]

    def process_frame(frame, eye_detector):
        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect the eyes in the frame
        eyes = eye_detector.detectMultiScale(gray, 1.3, 5)

        # Find the center of the eyes
        eye_centers = []
        for eye in eyes:
            eye_center = (eye[0] + eye[2] // 2, eye[1] + eye[3] // 2)
            eye_centers.append(eye_center)

        if len(eye_centers) >= 2:
            # Calculate the distance between the eyes
            eye_distance = np.linalg.norm(np.array(eye_centers[0]) - np.array(eye_centers[1]))

            # Calculate the percentage of how much the user is looking into the camera
            gaze_percentage = eye_distance / (frame.shape[1] // 2)
        else:
            gaze_percentage = -1  # Default value if eyes are not detected

        return gaze_percentage

    cap = cv2.VideoCapture(video_path)
    gaze_percentages = []

    with ThreadPoolExecutor(max_workers=len(cascade_classifiers)) as executor:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Process each frame with different Haar Cascade classifiers in parallel
            results = list(executor.map(process_frame, [frame] * len(cascade_classifiers), cascade_classifiers))

            # Use the results as needed (e.g., take the average)
            average_gaze = sum(results) / len(results)
            gaze_percentages.append(average_gaze)

    cap.release()

    if gaze_percentages:
        final_output = (sum(gaze_percentages) / len(gaze_percentages)) * 100 * -1
        print(f"Average Gaze Percentage: {final_output:.2f}")
        return final_output
    else:
        print("No gaze data available.")
        return jsonify({"error": "No gaze data available."})

In [8]:
def check_happiness(video_path):
    cap = cv2.VideoCapture(video_path)

    # Check if the camera is opened successfully
    if not cap.isOpened():
        print("Error: Could not open camera.")
        exit()


    total_happiness_percentage = 0
    total_sadness_percentage = 0
    frame_count = 0
    gaze_percentage = eye_tracking_method(video_path)
    while True:
        # Capture a single frame
        ret, frame = cap.read()

        # Break the loop if no more frames are available
        if not ret:
            break

        # Call your eye tracking function on the entire frame
        
        if(gaze_percentage < 0 or gaze_percentage == 100):
            return 1

        # Check if the person is looking (gaze_percentage > threshold, adjust the threshold accordingly)
        if gaze_percentage > 0:
            # Check if the frame is too dark or empty
            if np.mean(frame) < 10:
                happiness_percentage = 1
                sadness_percentage = 99
                return 1
            else:
                # Resize the frame to match the input size of your emotion classification model
                resized_frame = cv2.resize(frame, (256, 256))

                # Normalize pixel values
                resized_frame = resized_frame.astype('float32') / 255.0

                # Expand dimensions to create a batch (assuming the model expects a batch input)
                input_image = np.expand_dims(resized_frame, axis=0)

                # Make predictions using the emotion classification model
                predictions = emotion_model.predict(input_image)

                # Interpret the predictions
                # happiness_percentage, sadness_percentage = classify_emotion(predictions)

                print(f'Happiness Percentage: {happiness_percentage:.2f}%')
                print(f'Sadness Percentage: {sadness_percentage:.2f}%')

                # Accumulate the happiness and sadness percentages
                total_happiness_percentage += happiness_percentage
                total_sadness_percentage += sadness_percentage
                frame_count += 1

    # Release the video capture
    cap.release()

    if frame_count > 0:
        # Calculate the average happiness and sadness percentages
        average_happiness = total_happiness_percentage / frame_count
        average_sadness = total_sadness_percentage / frame_count

        print(f'Average Happiness Percentage: {average_happiness:.2f}%')
        print(f'Average Sadness Percentage: {average_sadness:.2f}%')
        print(gaze_percentage)  # Fix the format specifier here
        average_result = (average_happiness + gaze_percentage) / 2
        print(f'Average Result: {average_result:.2f}')
        return average_result
        # if(gaze_percentage < 0):
        #     return 1
        # else:
        #     return average_result
        # print('average'+(average_happiness+gaze_percentage)/2)
        # return (average_happiness+gaze_percentage)/2
        # Make a decision based on the average happiness
        # if average_happiness > 50:
        #     print("The person is happy on average!")
        # else:
        #     print("The person is not happy on average.")
    else:
        print("No frames available for processing.")
        return 1

In [9]:
def calculate_clarity():
    audio_file = "/home/jegathees5555/Documents/projects/AI_Interview_recruitz/backend/server/output.mp3"  # Replace with the path to your audio file
    
    # Load the audio file
    y, sr = librosa.load(audio_file)
    
    # Calculate the spectrogram
    spectrogram = np.abs(librosa.stft(y))
    # Calculate the spectral centroid
    spectral_centroid = librosa.feature.spectral_centroid(S=spectrogram)
    
    # Calculate the mean of the spectral centroid
    mean_centroid = np.mean(spectral_centroid)
    
    return mean_centroid/25

In [10]:
def calculate_boldness():
    audio_file = "/home/jegathees5555/Documents/projects/AI_Interview_recruitz/backend/server/output.mp3"  # Replace with the path to your audio file
    
    # Load the audio file
    y, sr = librosa.load(audio_file)
    
    # Calculate the spectrogram
    spectrogram = np.abs(librosa.stft(y))
    # Calculate the spectral contrast
    spectral_contrast = librosa.feature.spectral_contrast(S=spectrogram)
    
    # Calculate the mean of the spectral contrast
    mean_contrast = np.mean(spectral_contrast)
    
    return mean_contrast*3.2

In [11]:
# from backend.eye_track.eye_tracking import eye_tracking


@app.route('/upload_video_new', methods=['POST'])
def upload_frontend_video():
    try:
         # Remove the assumption of JSON data
        video_data = request.files['videoData']
        upload_video(video_data)
    except Exception as e:
        return jsonify({'error': str(e)}), 500
    
    try:
        result = eye_track_module()
    except Exception as e:
        return jsonify({'error': str(e)}), 500
    
    result_str = str(result)

    return jsonify({'message': 'Video uploaded successfully', 'result': result_str}), 200

In [12]:
@app.route('/get_result', methods=['GET'])
def getResult():
    try:
        video_path = '/home/jegathees5555/Documents/projects/AI_Interview_recruitz/backend/server/uploaded_video.webm'
        eye_contact = check_happiness(video_path)
        eye_contact = round(eye_contact,2)
        if(eye_contact == 1) :
            with app.app_context():
                # if(eye_contact == 1):
                return jsonify({ "eye_contact": eye_contact , "boldness" : 1, "clarity" : 1, "confidence" : 1, "overall" : 1})
                # else:
                #     return jsonify({ "eye_contact": eye_contact , "boldness" : boldness, "clarity" : clarity, "confidence" : confidence, "overall" : overall})
        # voice.voice_extraction_main()
        # voice_quality_analysis = voice_output.voice_quality()
        clarity = calculate_clarity()
        boldness = calculate_boldness()
        clarity = round(clarity,2)
        boldness = round(boldness,2)
        confidence = (eye_contact+clarity+boldness)/3
        confidence = round(confidence,2)
        overall = (eye_contact+boldness+clarity+confidence)//4
        overall = round(overall,2)
        # print("clarity :"+clarity + "confidence: "+confidence +" boldness : "+boldness +" eye_contact : "+eye_contact + "Overall : "+overall)
        with app.app_context():
            return jsonify({ "eye_contact": eye_contact , "boldness" : boldness, "clarity" : clarity, "confidence" : confidence, "overall" : overall})
    except Exception as e:
        return "Error occurred: " + str(e), 500

In [13]:
if __name__ == '__main__':
    # Set up ngrok and expose the Flask app
    ngrok_tunnel = ngrok.connect(5000)
    print('Public URL:', ngrok_tunnel.public_url)
    app.run(host='localhost', port=5000)

t=2024-01-26T18:34:48+0530 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=/home/jegathees5555/.config/ngrok/ngrok.yml legacy_path=/home/jegathees5555/.ngrok2/ngrok.yml


Public URL: https://b613-2409-40f4-38-7cba-1373-967b-ad8c-59bc.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5000
Press CTRL+C to quit


Average Gaze Percentage: 73.13
1/1 [==============================] - 0s 301ms/step


127.0.0.1 - - [26/Jan/2024 18:35:22] "GET /get_result HTTP/1.1" 500 -
